In [1]:
import pandas as pd
pd.options.plotting.backend='plotly'

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [7]:
loss_funcs = ['mse_20181210-20181216']
df = pd.DataFrame([])
for lf in loss_funcs:
    _df = pd.read_csv(f'jpl_v4_{lf}/errors.csv',index_col=0,usecols=[1,2,3,4])
    for col in _df.columns:
        _df.rename(columns={col:f'{lf}_{col}'},inplace=True)
    #_df['model'] = model
    df = pd.concat([df,_df],axis=1)

df.loc[:,[x+'_lstm' for x in loss_funcs]]#.plot();

mse_20181210-20181216_lstm
t                                              
2018-12-10 00:00:00                    1.180657
2018-12-10 01:00:00                    1.132077
2018-12-10 02:00:00                    1.179030
2018-12-10 03:00:00                    1.167168
2018-12-10 04:00:00                    1.202769
...                                         ...
2018-12-16 19:00:00                    0.690146
2018-12-16 20:00:00                    0.794540
2018-12-16 21:00:00                    0.819808
2018-12-16 22:00:00                    0.812582
2018-12-16 23:00:00                    0.833090

[168 rows x 1 columns]

In [5]:
loss_funcs = ['custom_loss','custom_mse2','mse']
df = pd.DataFrame([])
for lf in loss_funcs:
    _df = pd.read_csv(f'jpl_v4_{lf}/errors.csv',index_col=0,usecols=[1,2,3,4])
    for col in _df.columns:
        _df.rename(columns={col:f'{lf}_{col}'},inplace=True)
    #_df['model'] = model
    df = pd.concat([df,_df],axis=1)

df.loc[:,[x+'_lstm' for x in loss_funcs]]#.plot();

custom_loss_lstm  custom_mse2_lstm  mse_lstm
t                                                                
2019-12-12 00:00:00          1.385930          2.352678  2.421431
2019-12-12 01:00:00          1.512616          2.552807  2.603967
2019-12-12 02:00:00          1.702145          2.782472  2.813468
2019-12-12 03:00:00          1.765772          2.880867  2.877841
2019-12-12 04:00:00          1.860994          2.994767  2.962981
...                               ...               ...       ...
2020-01-15 20:00:00          1.485257          1.508196  1.700821
2020-01-15 21:00:00          1.823544          1.736881  2.083136
2020-01-15 22:00:00          2.244727          2.002960  2.355098
2020-01-15 23:00:00          2.700517          2.183143  2.673864
2020-01-16 00:00:00          2.727133          2.168596  2.562998

[841 rows x 3 columns]

In [10]:
onedrive_dir = 'C:/Users/woodj/OneDrive - Politecnico di Milano/'
#onedrive_dir = '/home/mjw/OneDrive'

measurements = pd.read_csv(onedrive_dir+'/MG2Lab Dataset/JPL_v4/all_JPL_v4.csv',
                   comment='#',index_col=0,parse_dates=True,usecols=[0,4])

In [11]:
def get_errors(feature='predicted_activepower_ev_1',loss_func='mse'):
    e = pd.DataFrame([])
    forecasts = pd.read_csv(f'jpl_v4_{loss_func}/forecasts.csv',index_col=1)
    for t in forecasts.index.unique():
        idx = pd.to_datetime(forecasts.loc[t,'timestamp_utc'])
        
        meas = measurements.loc[idx,'power'].values
        pred = forecasts.loc[t,feature].values
        
        e = pd.concat((e,pd.DataFrame({f'{t}':pred-meas},index=idx)),axis=1)
        
    return e
    

In [15]:
e = pd.DataFrame([])
_e = get_errors('persist','mse_20181210-20181216').resample('1h').mean().mean(axis=1) # not actually mse, predict has no loss func
e = pd.concat((e,_e),axis=1)
for lf in loss_funcs:
    _e = get_errors('predicted_activepower_ev_1',lf).resample('1h').mean().mean(axis=1) 
    e = pd.concat((e,_e),axis=1)
e.columns = ['persist']+loss_funcs
e.index = pd.to_datetime(e.index)
e['weekday'] = e.index.weekday

e

KeyError: "None of [DatetimeIndex(['2018-12-10 00:00:00', '2018-12-10 00:15:00',\n               '2018-12-10 00:30:00', '2018-12-10 00:45:00',\n               '2018-12-10 01:00:00', '2018-12-10 01:15:00',\n               '2018-12-10 01:30:00', '2018-12-10 01:45:00',\n               '2018-12-10 02:00:00', '2018-12-10 02:15:00',\n               ...\n               '2018-12-11 09:30:00', '2018-12-11 09:45:00',\n               '2018-12-11 10:00:00', '2018-12-11 10:15:00',\n               '2018-12-11 10:30:00', '2018-12-11 10:45:00',\n               '2018-12-11 11:00:00', '2018-12-11 11:15:00',\n               '2018-12-11 11:30:00', '2018-12-11 11:45:00'],\n              dtype='datetime64[ns]', name='times_utc-8', length=144, freq=None)] are in the [index]"

In [34]:
e = pd.DataFrame([])
_e = get_errors('persist','mse').resample('1h').mean().mean(axis=1) # not actually mse, predict has no loss func
e = pd.concat((e,_e),axis=1)
for lf in loss_funcs:
    _e = get_errors('predicted_activepower_ev_1',lf).resample('1h').mean().mean(axis=1) 
    e = pd.concat((e,_e),axis=1)
e.columns = ['persist']+loss_funcs
e.index = pd.to_datetime(e.index)
e['weekday'] = e.index.weekday

e

persist  custom_loss  custom_mse2        mse  weekday
2019-12-12 00:00:00  -0.469703    -0.465428    -0.191702  -0.426473        3
2019-12-12 01:00:00   0.000000     0.217209     0.007083   0.144086        3
2019-12-12 02:00:00   0.000000     0.181359     0.056669   0.010225        3
2019-12-12 03:00:00  -0.162117    -0.086977    -0.061975  -0.162088        3
2019-12-12 04:00:00  -0.066931    -0.535818    -0.496798  -0.538476        3
...                        ...          ...          ...        ...      ...
2020-01-17 07:00:00  17.532989    12.328699     6.989297  10.792440        4
2020-01-17 08:00:00  17.220980    15.370388     9.120874  10.560152        4
2020-01-17 09:00:00  15.667239    14.809040     8.921011   9.818334        4
2020-01-17 10:00:00  14.069732    13.232693     8.705898   9.140047        4
2020-01-17 11:00:00  14.183733    13.207026     9.493245  10.043529        4

[876 rows x 5 columns]

In [35]:
e.drop(columns=['weekday']).plot()

In [36]:
(e.custom_mse2 - e.mse).abs().mean() / e.mse.abs().max()

0.028157844705446435

In [37]:
(e.custom_loss - e.mse).abs().mean() / e.mse.abs().max()

0.03661870350214844

In [43]:
ewkdy = e[[0<=d<=3 for d in e.index.weekday]]

epeak = ewkdy[[7<=h<=13 for h in ewkdy.index.hour]]
epeak.mean()


persist        0.247473
custom_loss   -0.938958
custom_mse2   -1.481990
mse           -1.829421
weekday        1.571429
dtype: float64

In [44]:
epeak.abs().mean()

persist        2.531501
custom_loss    2.383152
custom_mse2    2.533104
mse            2.804849
weekday        1.571429
dtype: float64